In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
%matplotlib notebook
import plotly.express as px

df_cal = pd.read_csv("data/infectious-diseases-by-county-year-and-sex.csv")
df = df_cal[df_cal['County'] != 'CALIFORNIA'].copy()
df.head()

In [ ]:
diseases = df['Disease'].unique()

d_list = []
for d in diseases:
    tmp_df = df[df['Disease'] == d]
    total_tmp_df = tmp_df[tmp_df['Sex'] == 'TOTAL']
    d_list.append(total_tmp_df)

    total_sum = []
    for c in total_tmp_df['Year'].unique():
        #print(c, total_tmp_df[total_tmp_df['Year'] == c]['Cases'].sum())
        total_sum.append((c, total_tmp_df[total_tmp_df['Year'] == c]['Cases'].sum()))
    total_sum = pd.DataFrame(total_sum, columns=['Year', 'Cases'])
    
#     fig, ax = plt.subplots(figsize=(25,14))
#     total_sum.plot(x='Year', y='Cases', ax=ax)
#     plt.title(d)
#     plt.savefig('images/'+ d.replace('.', '_'))
#     plt.close()



In [ ]:
county_info = pd.read_csv("data/county_info.csv", sep=';', dtype={'FIPS': object})

county_info['Per capita income'] = county_info['Per capita income'].apply(lambda x: x.replace('$', ''))
county_info['Median household income'] = county_info['Median household income'].apply(lambda x: x.replace('$', ''))
county_info['Median family income'] = county_info['Median family income'].apply(lambda x: x.replace('$', ''))

county_info['Per capita income'] = county_info['Per capita income'].astype(int)
county_info['Median household income'] = county_info['Median household income'].astype(int)
county_info['Median family income'] = county_info['Median family income'].astype(int)
county_info['FIPS'] = county_info['FIPS'].apply(lambda x: "06"+x)

county_info.tail()

In [ ]:
half_size = int(len(county_info)/2)
width = 0.8


import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Per capita', x=county_info['County'], y=county_info['Per capita income']),
    go.Bar(name='Household', x=county_info['County'], y=county_info['Median household income']),
    go.Bar(name='Family', x=county_info['County'], y=county_info['Median family income'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [ ]:
maxlist = []
max10list = [[0,'temp']]
siz = 10


for d in d_list:
    tup = [d['Cases'].max(), d['Disease'].iloc[0]]
    maxlist.append(tup)

    contains = False
    for i in range(len(max10list)):
        if tup[0]==max10list[i][1]:
            contains = True
            break
    
    if not contains:
        for i in range(len(max10list)):
            t = max10list[i]
            val = t[0]
            if tup[0]>val:
#                 print(tup, i)
                max10list.insert(i, tup)
                break
            if i>siz:
                max10list = max10list[0:siz]
                break


big_d_list = []
for i in range(len(max10list)):
    #print(max10list[i])
    big_d_list.append(max10list[i][1])

virus_list = []
virusses = []
for d in d_list:
    if 'irus' in d['Disease'].iloc[0]:
        virus_list.append(d)
        virusses.append(d['Disease'].iloc[0])
        if d['Disease'].iloc[0] not in big_d_list:
            big_d_list.append(d['Disease'].iloc[0])
        
big_d_list

test_list = []
for d in d_list:
    if d['Disease'].iloc[0] in big_d_list:
        test_list.append(d)
        
big_d_list

In [ ]:
min_income = county_info['Per capita income'].min()
max_income = county_info['Per capita income'].max()
diff = max_income - min_income
third = min_income + diff/3
sixth = min_income + diff/3 * 2

lower_class = (min_income, third)
middle_class = (third, sixth)
upper_class = (sixth, max_income)

print(lower_class)
print(middle_class)
print(upper_class)

def get_class(income):
    if income > sixth:
        return "Upper Class"
    elif income > third:
        return "Middle Class"
    else:
        return "Lower Class"

In [ ]:
import os

if not os.path.exists("images"):
    os.mkdir("images")

# for i in range(len(test_list)):
#     disease = test_list[i]
#     counties = disease['County'].unique()
#     c_sum = []
#     for c in counties:
#         if c != 'CALIFORNIA':
#             pop = county_info[county_info['County'] == c.title()]['Population'].values
#             den = county_info[county_info['County'] == c.title()]['Population Density'].values
#             county_class = get_class(county_info[county_info['County'] == c.title()]['Per capita income'].values)
#             c_sum.append((c, 
#                           disease[disease['County'] == c]['Cases'].sum(), 
#                           disease[disease['County'] == c]['Rate'].sum(), 
#                           pop, 
#                           den,
#                           county_class))

#     c_sum = pd.DataFrame(c_sum, columns=['County', 'Sum', 'Rate', 'Population', 'Density', 'Class'])
#     fig = px.scatter(c_sum, 
#                      x='Rate', 
#                      y='Sum', 
#                      size=list(c_sum['Population'].values), 
#                      color='Class')
#     fig.update_layout(title=disease['Disease'].iloc[0])
#     fig.write_image("images/" + disease['Disease'].iloc[0] + ".svg")
#     fig.close()


In [ ]:
import plotly.figure_factory as ff
import plotly
print(len(test_list))
for i in range(len(test_list)):
    for year in range(2001, 2019):
        test = test_list[i]
        test = test[test['Year'] == year]
        test = test[['County', 'Cases']]
        test['FIPS'] = 0
        fips = []
        for index, row in test.iterrows():
            t_fip = county_info[county_info['County'] == row['County'].title()]['FIPS'].values[0]
            fips.append(t_fip)

        fips = pd.Series(fips)
        test = test.reset_index()
        test = test.drop('index', axis=1)
        test['FIPS'] = fips

        values = test['Cases'].tolist()
        fips = test['FIPS'].tolist()

        colorscale = [
            'rgb(193, 193, 193)',
            'rgb(239,239,239)',
            'rgb(195, 196, 222)',
            'rgb(144,148,194)',
            'rgb(101,104,168)',
            'rgb(65, 53, 132)'
        ]

        fig = ff.create_choropleth(fips=fips, values=values, scope=['CA'])
        fig.update_layout(title=test_list[i]['Disease'].iloc[0])
        fig.layout.template = None
        path = "images/maps/"+ test_list[i]['Disease'].iloc[0]
        if not os.path.exists(path):
            os.mkdir(path)
        path = path + "/" + test_list[i]['Disease'].iloc[0] + str(year) + ".svg"
        print(path)
        fig.write_image(path)
        del fig

# Rik's plotting

In [ ]:
counties = df['County'].unique()
c_list = []

# for c in counties:
#     tmp_df = df[df['County'] == c]
#     total_tmp_df = tmp_df[tmp_df['Sex'] == 'TOTAL']
#     c_list.append(total_tmp_df)

#     total_sum = []
#     for y in total_tmp_df['Year'].unique():
#         #print(c, total_tmp_df[total_tmp_df['Year'] == c]['Cases'].sum())
#         total_sum.append((y, total_tmp_df[total_tmp_df['Year'] == y]['Cases'].sum()))
#     total_sum = pd.DataFrame(total_sum, columns=['Year', 'Cases'])
    
#     fig, ax = plt.subplots(figsize=(25,14))
#     total_sum.plot(x='Year', y='Cases', ax=ax)
#     plt.title(c)
# #     plt.savefig('images/'+ d.replace('.', '_'))

In [ ]:
cd_list = []
sumlist = []
# for c in counties:
    
#     for d in diseases:
#         tmp = df[df['County'] == c]
#         tmp_df = tmp[tmp['Disease'] == d]
#         total_tmp_df = tmp_df[tmp_df['Sex'] == 'TOTAL']
#         cd_list.append(total_tmp_df)

#         total_sum = []
#         for y in total_tmp_df['Year'].unique():
#             #print(c, total_tmp_df[total_tmp_df['Year'] == c]['Cases'].sum())
#             total_sum.append((y, total_tmp_df[total_tmp_df['Year'] == y]['Cases'].sum()))
#         total_sum = pd.DataFrame(total_sum, columns=['Year', 'Cases'])

#         fig, ax = plt.subplots(figsize=(25,14))
#         total_sum.plot(x='Year', y='Cases', ax=ax, label=d)
#         plt.title(c)
#         plt.show()
#         #     plt.savefig('images/'+ d.replace('.', '_'))

In [ ]:
cd_list = []
sumlist = []
# for c in counties:
    
#     for d in diseases:
#         tmp = df[df['County'] == c]
#         tmp_df = tmp[tmp['Disease'] == d]
#         total_tmp_df = tmp_df[tmp_df['Sex'] == 'TOTAL']
#         cd_list.append(total_tmp_df)

#         total_sum = []
#         for y in total_tmp_df['Year'].unique():
#             #print(c, total_tmp_df[total_tmp_df['Year'] == c]['Cases'].sum())
#             total_sum.append((y, total_tmp_df[total_tmp_df['Year'] == y]['Cases'].sum()))
#         total_sum = pd.DataFrame(total_sum, columns=['Year', 'Cases'])
#         l = [total_sum, d]
        
#         sumlist.append(l)
#     fig, ax = plt.subplots(figsize=(25,14))
    
#     for sum in sumlist:
#         label = sum[1]
#         sum[0].plot(x="Year", y="Cases", ax=ax, label=label)

#     #         total_sum.plot(x='Year', y='Cases', ax=ax, label=d)
#     plt.title(c)
#     plt.show()
#         #     plt.savefig('images/'+ d.replace('.', '_'))

In [ ]:
for c in counties: 
    for d in diseases:
        tmp = df[df['County'] == c]
        tmp_df = tmp[tmp['Disease'] == d]
        total_tmp_df = tmp_df[tmp_df['Sex'] == 'TOTAL']
        cd_list.append(total_tmp_df)

        total_sum = []
        for y in total_tmp_df['Year'].unique():
            #print(c, total_tmp_df[total_tmp_df['Year'] == c]['Cases'].sum())
            total_sum.append((y, total_tmp_df[total_tmp_df['Year'] == y]['Rate'].sum()))
        total_sum = pd.DataFrame(total_sum, columns=['Year', 'Rate'])
        l = [total_sum, d]
        
        sumlist.append(l)
    fig, ax = plt.subplots(figsize=(25,14))
    
    for sum in sumlist:
        label = sum[1]
        sum[0].plot(x="Year", y="Rate", ax=ax, label=label)

    #         total_sum.plot(x='Year', y='Cases', ax=ax, label=d)
    plt.title(c)
    plt.show()
        #     plt.savefig('images/'+ d.replace('.', '_'))

In [ ]:
d_list[0]

# Dit is animatie shit

In [ ]:
# amebiasis = d_list[0]
# print(f" There are {len(amebiasis['Year'].unique())} years")

# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=20, metadata=dict(artist='Me'), bitrate=1800)

# fig, ax = plt.subplots(figsize=(10,6))
# counties = amebiasis['County'].unique()

# def animate_county(i):
#     ax.clear()
#     county = counties[i]
#     data = amebiasis[amebiasis['County'] == county] #select data range
#     plt.xlim(2001, 2018)
#     plt.xlabel('County',fontsize=20)
#     plt.ylabel('Rate',fontsize=20)
#     plt.ylim(0, 20)
#     plt.title(county)
#     p = sns.lineplot(x=data['Year'], y=data['Rate'], data=data, ax=ax)
#     #p.tick_params(labelsize=17)
#     #plt.setp(p.lines,linewidth=7)
    
# ani = animation.FuncAnimation(fig, animate_county, frames=59, repeat=False)
# plt.show()

In [ ]:
# import random

# d_num = 0
# c_num = 0


# disease = d_list[d_num].sort_values(by=['Rate'], ascending=False)
# counties = disease['County'].unique()
# county = counties[c_num]
# colors = [(random.random(), random.random(), random.random()) for x in counties]

# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=20, metadata=dict(artist='Me'), bitrate=1800)

# fig2, ax2 = plt.subplots(figsize=(10,6))
# years = range(2001,2018)

# def animate_years(i):
    
#     index = i // 19
#     county = counties[index]
#     data = disease[disease['County'] == county].sort_values(by=['Year']).iloc[:i%19]
#     plt.xlim(2001, 2018)
#     plt.ylim(0, 20)
#     plt.title(county + " " + disease['Disease'].iloc[0])
#     p = sns.lineplot(x='Year', y='Rate', data=data, ax=ax2, color = colors[index])
#     plt.setp(p.lines,linewidth=2)

    
# ani = animation.FuncAnimation(fig2, animate_years, frames=19, repeat=False, interval=1)
# plt.show()